In [1]:
# first set up which gpu to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [2]:
# import libraries
import numpy as np
from termcolor import colored, cprint
# for display
from IPython.display import Image as ipy_image
from IPython.display import display

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils

from datasets.dataloader import CreateDataLoader, get_data_generator
from models.base_model import create_model
from utils.util_3d import render_sdf, render_mesh, sdf_to_mesh, save_mesh_as_gif

%load_ext autoreload
%autoreload 2

/root/miniconda3/envs/sdfusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import VQVAEOpt

seed = 2023
opt = VQVAEOpt(seed=seed)
device = opt.device

[*] VQVAETestOpt initialized.


In [15]:
vqvae_path = "/root/autodl-tmp/SDFusion/saved_ckpt/vqvae-snet-all.pth"
opt.isTrain = False
opt.init_model_args(ckpt_path=vqvae_path, vq_cfg='configs/vqvae_snet.yaml')
vqvae = create_model(opt)

Working with z of shape (1, 3, 16, 16, 16) = 12288 dimensions.
[*] weight successfully load from: /root/autodl-tmp/SDFusion/saved_ckpt/vqvae-snet-all.pth
[*] Model has been created: VQVAE-Model


In [ ]:
obj_path = "/root/autodl-tmp/SDFusion/data/ShapeNet/ShapeNetV1processed/04256520/1a04e3eab45d6f4a2a6e76a921d3343a/models/model_normalized.obj"

# turn obj to sdf
